# scheduling after getting Scheduling_priority

In the following algorithm we are considering the 5 factors: {Asset_criticality, Urgency, User_status, Ageing, man_hrs}
    If we have jobs with same scheduling priority then we prioritize ORDER_REL, 
    if again we get tie then we schedule old jobs first, 
    if again we get tie we shedule less work_duraion jobs first.
    

In [ ]:
import pandas as pd

def read_excel(file_path):
    # Read the Excel file
    df = pd.read_excel(file_path)
    return df

def find_best_combination(tasks, day_limit, week_limit):
    tasks_sorted = tasks.sort_values(by=['Scheduling_priority', 'User_status', 'Ageing', 'man_hrs'], 
                           ascending=[True, False, False, True]).to_dict('records')
    days = [[]]
    daily_sums = [0]

    for task in tasks_sorted:
        placed = False
        for i in range(len(days)):
            if daily_sums[i] + task['man_hrs'] <= day_limit:
                days[i].append(task)
                daily_sums[i] += task['man_hrs']
                placed = True
                break
        if not placed:
            days.append([task])
            daily_sums.append(task['man_hrs'])

    # Ensure weekly constraint
    weeks = []
    week = []
    week_sum = 0
    for day, daily_sum in zip(days, daily_sums):
        if week_sum + daily_sum > week_limit:
            weeks.append(week)
            week = []
            week_sum = 0
        week.append(day)
        week_sum += daily_sum
    if week:
        weeks.append(week)

    return weeks

def write_schedule_to_excel(file_path, schedule):
    writer = pd.ExcelWriter(file_path, engine='openpyxl')

    for week_index, week in enumerate(schedule):
        week_data = []
        for day_index, day in enumerate(week):
            for task in day:
                task['Week'] = week_index + 1
                task['Day'] = day_index + 1
                week_data.append(task)
        df_week = pd.DataFrame(week_data)
        df_week.to_excel(writer, sheet_name=f'Week_{week_index+1}', index=False)

    writer.save()

#main function
def main(input_file, output_file):
    # Read the input Excel file
    tasks_df = read_excel(input_file)

    # Add 'man_hrs' column if not present
    if 'man_hrs' not in tasks_df.columns:
        tasks_df['man_hrs'] = tasks_df['job_duration'] * tasks_df['man_power']

    # Define limits
    day_limit = 80
    week_limit = 560

    # Find the best combination
    schedule = find_best_combination(tasks_df, day_limit, week_limit)

    # Write the schedule to the output Excel file
    write_schedule_to_excel(output_file, schedule)

# Example usage
input_file = 'Vedanta_job_data.xlsx'  # Replace with your input Excel file path
output_file = 'prioritized_jobs.xlsx'  # Replace with your desired output Excel file path

main(input_file, output_file)

# If input Excel file has many sheets, and we want to use one of them, use the following code with sheet_name parameter in the pd.read_excel function:

In [2]:
import pandas as pd

def read_excel(file_path, sheet_name):
    # Read the specific sheet from the Excel file
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    return df

def find_best_combination(tasks, day_limit, week_limit):
    tasks_sorted = tasks.sort_values(by=['Scheduling_priority', 'User_status', 'Ageing', 'man_hrs'], 
                           ascending=[True, False, False, True]).to_dict('records')
    days = [[]]
    daily_sums = [0]

    for task in tasks_sorted:
        placed = False
        for i in range(len(days)):
            if daily_sums[i] + task['man_hrs'] <= day_limit:
                days[i].append(task)
                daily_sums[i] += task['man_hrs']
                placed = True
                break
        if not placed:
            days.append([task])
            daily_sums.append(task['man_hrs'])

    # Ensure weekly constraint
    weeks = []
    week = []
    week_sum = 0
    for day, daily_sum in zip(days, daily_sums):
        if week_sum + daily_sum > week_limit:
            weeks.append(week)
            week = []
            week_sum = 0
        week.append(day)
        week_sum += daily_sum
    if week:
        weeks.append(week)

    return weeks

def write_schedule_to_excel(file_path, schedule):
    writer = pd.ExcelWriter(file_path, engine='openpyxl')

    for week_index, week in enumerate(schedule):
        week_data = []
        for day_index, day in enumerate(week):
            for task in day:
                task['Week'] = week_index + 1
                task['Day'] = day_index + 1
                week_data.append(task)
        df_week = pd.DataFrame(week_data)
        df_week.to_excel(writer, sheet_name=f'Week_{week_index+1}', index=False)

    writer.save()

# Main function
def main(input_file, output_file):
    
    # Define the sheet name to read
    sheet_name = 'WO_with_SchPr'

    # Read the input Excel file
    tasks_df = read_excel(input_file, sheet_name)

    # Add 'man_hrs' column if not present
    if 'man_hrs' not in tasks_df.columns:
        tasks_df['man_hrs'] = tasks_df['work_duration'] * tasks_df['man_power']

    # Define limits
    day_limit = 200
    week_limit = 1400

    # Find the best combination
    schedule = find_best_combination(tasks_df, day_limit, week_limit)

    # Write the schedule to the output Excel file
    write_schedule_to_excel(output_file, schedule)

# Example usage
input_file = 'Red2_July_W4_Plan.xlsx'  # Replace with your input Excel file path
output_file = 'Red2_July_W4_Scheduled_Plan.xlsx'  # Replace with your desired output Excel file path

main(input_file, output_file)


C:\Users\Vermas\AppData\Local\Temp\ipykernel_35112\795958958.py:55: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
